In [1]:
import chromedriver_autoinstaller as chromedriver
chromedriver.install() 

'/home/ndjogo/anaconda3/lib/python3.9/site-packages/chromedriver_autoinstaller/108/chromedriver'

In [2]:
from urllib.request import urlopen 

# Creating the class for website extraction 
class Scrapedata():
    def __init__(self,url):
        #initialising the url attribute 
        self.url = url
        print(f"The scraper object has been initialised with url {self.url}")
        print("We are scraping the webpage....")
        #opening the url
        page = urlopen(self.url)
        print("Scraping complete, the scraped page is printed below")
        #reading the html and decoding it
        html = page.read()
        html = html.decode("utf-8")
        #saving the html file 
        with open("scraped_page.html", "w") as file:
            file.write(html)


        print(html)


scraperobject = Scrapedata("https://www.rottentomatoes.com/browse/movies_at_home/genres:horror?page=1") 


The scraper object has been initialised with url https://www.rottentomatoes.com/browse/movies_at_home/genres:horror?page=1
We are scraping the webpage....
Scraping complete, the scraped page is printed below
<!DOCTYPE html>
<html lang="en" dir="ltr" xmlns="http://www.w3.org/1999/xhtml" prefix="fb: http://www.facebook.com/2008/fbml og: http://opengraphprotocol.org/schema/">
    <head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#">
        
            <!-- OneTrust Cookies Consent Notice start for rottentomatoes.com -->
            <script src="https://cdn.cookielaw.org/consent/7e979733-6841-4fce-9182-515fac69187f/otSDKStub.js"
                type="text/javascript"
                charset="UTF-8"
                data-domain-script="7e979733-6841-4fce-9182-515fac69187f"
                integrity="sha384-WEHwEli88wqOiQd913F1utFZiwisa8XhCkbjLnbKEpFa/WbFcPKeGg7h4fdsv0Z/"
                crossorigin="anonymous">
            </script>
            <sc

In [3]:
scraperobject

In [5]:
!pip install selenium 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 8.2 MB/s eta 0:00:000:00:010:00:01:01
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 8.5 MB/s eta 0:00:00 MB/s eta 0:00:01
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00


In [2]:

from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time 

In [3]:
driver = webdriver.Chrome()

In [27]:
driver.get("https://www.rottentomatoes.com/browse/movies_at_home/genres:horror?page=1")

In [34]:
# len(driver.find_elements(by = By.CLASS_NAME, value = 'js-tile-link'))

In [35]:
# for pages in range(10):
#     next_button = driver.find_element(by = By.XPATH, value = '//*[@id="main-page-content"]/div/div[5]/button')
#     next_button.click() 
#     time.sleep(4)
#     print(len(driver.find_elements(by = By.CLASS_NAME, value = 'js-tile-link')))

In [28]:
try:
    cookie_accept = driver.find_element(by = By.XPATH, value = '//*[@id="onetrust-accept-btn-handler"]')
    cookie_accept.click()
except:
    pass


In [29]:
num_pages = 5
for pages in range(num_pages - 1): 
    next_button = driver.find_element(by = By.XPATH, value = '//*[@id="main-page-content"]/div/div[5]/button')
    next_button.click()
    time.sleep(4)
    
movies = driver.find_elements(by = By.CLASS_NAME, value = 'js-tile-link')

    

In [24]:
# for item in movies:
#     if "WATCH THE TRAILER" in item.text:
#         print(item.text.split('\n')[1:])
#     else:
#         print(item.text.split('\n'))
        

    

['--', '--', 'The Sixth Secret', 'Streaming Jan 10, 2023']
['--', '--', 'Human Resources', 'Streaming Jan 10, 2023']
['--', '--', 'Those Who Call', 'Streaming Jan 10, 2023']
['96%', '--', 'Sick', 'Streaming Jan 13, 2023']
['90%', '--', 'The Offering', 'Streaming Jan 13, 2023']
['--', '--', 'Night of the Bastard', 'Streaming Jan 13, 2023']
['89%', '76%', 'The Menu', 'Streaming Jan 3, 2023']
['63%', '32%', 'White Noise', 'Streaming Dec 30, 2022']
['92%', '71%', 'Barbarian', 'Streaming Oct 25, 2022']
['76%', '80%', 'Candy Land', 'Streaming Jan 6, 2023']
['83%', '69%', 'Nope', 'Streaming Nov 18, 2022']
['94%', '75%', 'X', 'Streaming Apr 14, 2022']
['80%', '77%', 'Smile', 'Streaming Nov 15, 2022']
['96%', '73%', 'The Innocents', 'Streaming May 13, 2022']
['97%', '56%', 'Hellbender', 'Streaming Feb 24, 2022']
['90%', '69%', 'Hereditary', 'Streaming Aug 21, 2018']
['91%', '82%', 'Pearl', 'Streaming Oct 25, 2022']
['89%', '46%', 'Nanny', 'Streaming Dec 16, 2022']
['17%', '69%', 'Prey for the D

In [31]:
n_movies = 15*num_pages**2+15*num_pages+30

movie_information = []
for movie in range(1,7):
    movie_dict = {}
    movie_link = driver.find_element(by = By.XPATH, value = f'//*[@id="main-page-content"]/div/div[3]/div/div/a[{movie}]/tile-dynamic/div/span[1]')
    movie_link.click()
    time.sleep(4)
    try:
        movie_info = driver.find_element(by = By.XPATH, value = '//*[@id="topSection"]/div[1]/score-board/p')
        movie_dict['movie_info'] = movie_info.text   
    except:
        movie_dict['movie_info'] = 'Not available'

    if "WATCH THE TRAILER" in movies[movie - 1].text:
        movie_dict["movie_name"] = movies[movie - 1].text.split('\n')[3]
        movie_dict["critics_score"] = movies[movie - 1].text.split('\n')[1]
        movie_dict["audience_score"] = movies[movie - 1].text.split('\n')[2]
    else:
        movie_dict["movie_name"] = movies[movie - 1].text.split('\n')[2]
        movie_dict["critics_score"] = movies[movie - 1].text.split('\n')[0]
        movie_dict["audience_score"] = movies[movie - 1].text.split('\n')[1]
        
    if movie_dict["audience_score"] == "--" : 
        movie_extra_info = driver.find_element(by = By.XPATH, value = '//*[@id="mainColumn"]/section[2]/div/div')
    else:
        movie_extra_info = driver.find_element(by = By.XPATH, value = '//*[@id="mainColumn"]/section[3]/div/div')
    
    movie_text = movie_extra_info.text.split('\n')
    print(movie_text)
    movie_dict['Description'] = movie_text[0]
    for i in range(1,len(movie_text)):
        temp_split = movie_text[i].split(': ')
        movie_dict[temp_split[0]] = temp_split[1]
    
    movie_information.append(movie_dict)
    driver.back()
    time.sleep(4)

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=108.0.5359.124)
Stacktrace:
#0 0x55a167637563 <unknown>
#1 0x55a1673f6667 <unknown>
#2 0x55a1673fa0b8 <unknown>
#3 0x55a1673f9e82 <unknown>
#4 0x55a1673fa18c <unknown>
#5 0x55a16742d05e <unknown>
#6 0x55a167453122 <unknown>
#7 0x55a167427c0f <unknown>
#8 0x55a1674532ee <unknown>
#9 0x55a16746aa7c <unknown>
#10 0x55a167452ef3 <unknown>
#11 0x55a16742649e <unknown>
#12 0x55a1674275ae <unknown>
#13 0x55a167686fde <unknown>
#14 0x55a16768a4c4 <unknown>
#15 0x55a16766d78e <unknown>
#16 0x55a16768b393 <unknown>
#17 0x55a167660665 <unknown>
#18 0x55a1676ac108 <unknown>
#19 0x55a1676ac296 <unknown>
#20 0x55a1676c7183 <unknown>
#21 0x7f3bfdafdb43 <unknown>


In [20]:
movie_information

[{'movie_info': '2023, Comedy/Horror, 1h 29m',
  'critics_score': '1000',
  'audience_score': '1000'},
 {'movie_info': '2021, Horror/Mystery & thriller, 1h 48m',
  'critics_score': '1000',
  'audience_score': '1000'},
 {'movie_info': '2022, Horror/Mystery & thriller, 1h 33m',
  'critics_score': '1000',
  'audience_score': '1000'},
 {'movie_info': '2022, Horror/Mystery & thriller, 1h 23m',
  'critics_score': '1000',
  'audience_score': '1000'},
 {'movie_info': '2022, Horror, 1h 22m',
  'critics_score': '1000',
  'audience_score': '1000'},
 {'movie_info': '2022, Horror/Mystery & thriller, 1h 47m',
  'critics_score': '1000',
  'audience_score': '1000'}]

In [14]:
#driver.get("https://www.rottentomatoes.com/m/the_menu")
test_selector = driver.find_element(by = By.XPATH, value = '//*[@id="mainColumn"]/section[3]/div/div')
movie_text = test_selector.text.split('\n')
movie_info_dict = {}
movie_info_dict['Description'] = movie_text[0]
for i in range(1,len(movie_text)):
    temp_split = movie_text[i].split(': ')
    movie_info_dict[temp_split[0]] = temp_split[1]
    
 
print(movie_info_dict)
    

{'Description': 'A couple (Anya Taylor-Joy and Nicholas Hoult) travels to a coastal island to eat at an exclusive restaurant where the chef (Ralph Fiennes) has prepared a lavish menu, with some shocking surprises.', 'Rating': 'R (Some Sexual References|Language Throughout|Strong Violent Content)', 'Genre': 'Horror, Mystery & Thriller, Comedy', 'Original Language': 'English', 'Director': 'Mark Mylod', 'Producer': 'Adam McKay, Betsy Koch, Will Ferrell', 'Writer': 'Seth Reiss, Will Tracy', 'Release Date (Theaters)': 'Nov 18, 2022  Wide', 'Release Date (Streaming)': 'Jan 3, 2023', 'Box Office (Gross USA)': '$37.6M', 'Runtime': '1h 47m', 'Distributor': 'Searchlight Pictures', 'Sound Mix': 'Dolby Atmos, Dolby Digital', 'Aspect Ratio': 'Scope (2.35:1)'}


In [38]:
'//*[@id="movie-cast"]/div'

'//*[@id="mainColumn"]/section[2]/div/div'


'//*[@id="mainColumn"]/section[2]/div/div'


'//*[@id="mainColumn"]/section[3]/div/div'


'//*[@id="mainColumn"]/section[2]/div/div'

--
--
The Sixth Secret
Streaming Jan 10, 2023
--
--
Human Resources
Streaming Jan 10, 2023
WATCH THE TRAILER FOR THE OFFERING
89%
--
The Offering
Streaming Jan 13, 2023
95%
--
Sick
Streaming Jan 13, 2023
--
--
Night of the Bastard
Streaming Jan 13, 2023
WATCH THE TRAILER FOR THE MENU
89%
76%
The Menu
Streaming Jan 3, 2023
WATCH THE TRAILER FOR WHITE NOISE
63%
32%
White Noise
Streaming Dec 30, 2022
WATCH THE TRAILER FOR BARBARIAN
92%
71%
Barbarian
Streaming Oct 25, 2022
WATCH THE TRAILER FOR NOPE
83%
69%
Nope
Streaming Nov 18, 2022
WATCH THE TRAILER FOR CANDY LAND
76%
79%
Candy Land
Streaming Jan 6, 2023
WATCH THE TRAILER FOR X
94%
75%
X
Streaming Apr 14, 2022
WATCH THE TRAILER FOR SMILE
80%
77%
Smile
Streaming Nov 15, 2022
WATCH THE TRAILER FOR THE INNOCENTS
96%
73%
The Innocents
Streaming May 13, 2022
WATCH THE TRAILER FOR HELLBENDER
97%
56%
Hellbender
Streaming Feb 24, 2022
WATCH THE TRAILER FOR HEREDITARY
90%
69%
Hereditary
Streaming Aug 21, 2018
WATCH THE TRAILER FOR PEARL
91%
82%


In [ ]:
"https://www.rottentomatoes.com/browse/movies_at_home/genres:horror?page=5"
"https://www.rottentomatoes.com/m/the_menu"
"https://www.rottentomatoes.com/m/the_pale_blue_eye"
"https://www.rottentomatoes.com/m/were_all_going_to_the_worlds_fair"

In [36]:
/html/body/div[3]/main/div[1]/div/div[3]/div/div/a[1]/tile-dynamic/div/span[1]
/html/body/div[3]/main/div[1]/div/div[3]/div/div/a[2]/tile-dynamic/div/span[1]
//*[@id="main-page-content"]/div/div[3]/div/div/a[3]/tile-dynamic/div/span[1]

NameError: name 'html' is not defined